In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, execute, QuantumCircuit
# Loading your IBM Q account(s)
import pylab
import numpy as np
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
import time
from qiskit.tools.monitor import job_monitor
from datetime import datetime
import sys
sys.path.insert(0, './')
from utility import *

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# load account

In [2]:
IBMQ.load_account()
#provider = IBMQ.get_provider(hub = "ibm-q-research-2", group='uni-georgia-1', project='main')
provider = IBMQ.get_provider(hub = "ibm-q", group='open', project='main')
backend = provider.get_backend('ibmq_qasm_simulator')#('ibmq_manila')
backend.configuration()

ibmqfactory.load_account:WARNING:2022-06-24 02:46:48,617: Credentials are already in use. The existing account in the session will be replaced.


# experiment 

## K = 7; s = 5; snr =3

In [3]:

def QAS_linearReg(Input):
    Y_train= Input["Y_train"]
    X_train= Input["X_train"]
    Y_test= Input["Y_test"]
    X_test= Input["X_test"]
    initial_index= Input["initial_index"]
    learn_rate=Input["learn_rate"]
    m = 1
    Grover_iters = 1
    p = X_train.shape[1]
    
    initial_Bin = "0"*p
    for index in initial_index:
        initial_Bin[index]=1
    score_initial = score(initial_Bin,Y_train,X_train,Y_test,X_test)
    
    ScoresList = []
    Binlist = []
    for i in range(2**p):
        Bin = bin(i).replace('0b', '')
        Binlist.append(list(Bin.rjust(p,'0')))
        ScoresList.append(score(Bin.rjust(p,'0'),Y_train,X_train,Y_test,X_test))
    ScoresList = np.array(ScoresList)
    Selected_Scores =[]

    time_used = 0
    while (m < 15*learn_rate*np.log(p)) and sum(score_initial>ScoresList) != 0:
        iter_time = int(0.25*np.pi*learn_rate**(-m/2))+1
        m = m+1
        OracleState_index = np.where(ScoresList<score_initial)[0]
        oracles_bin = [Binlist[k] for k in OracleState_index]

        new_Bin,time = Grover(oracles_bin,p,iter_time,backend)
        print(new_Bin)
        time_used = time_used+time.total_seconds()
        state_index = int(new_Bin,2)
        score_new = ScoresList[state_index]
        if score_initial> score_new:
            selected_index = state_index
            score_initial = score_new
            m = 1
        Selected_Scores.append(score_initial)
    print("propose state is %d"%selected_index)
    return([selected_index,Selected_Scores,time_used])

In [ ]:
N=200
n=100
p=7
s=5
snr=3
rho=0.25

B = np.zeros((p,))
B[0:s]=1
Sigma = np.zeros((p,p))
for i in range(p):
    for j in range(p):
        Sigma[i,j] = rho**abs(i-j)
mu = np.zeros((p,))

Selected_states =[]
res7_3 = []
for rep in range(100):
    np.random.seed(101+rep)
    X = np.random.multivariate_normal(mu, Sigma, N)+1
    epsion = np.random.normal(0,1,N)

    Y = X.dot(B)+np.sqrt(B.dot(Sigma).dot(B)/snr)*epsion
    Y_train = Y[0:n]
    X_train = X[0:n]
    Y_test = Y[(n+1):N]
    X_test = X[(n+1):N]
    Input = dict({"Y_train":Y_train,"X_train":X_train,"Y_test":Y_test,"X_test":X_test,\
              "initial_index":[],"learn_rate":0.65})
    experiment_res = QAS_linearReg(Input)
    Selected_states.append(experiment_res[0])
    res7_3.append(experiment_res)

Job Status: job has successfully run
1111111
Job Status: job has successfully run
0111111
Job Status: job has successfully run
1111101
Job Status: job has successfully run
1100010
Job Status: job has successfully run
1101110
Job Status: job has successfully run
1000100
Job Status: job has successfully run
1111110
Job Status: job has successfully run
0011111
Job Status: job has successfully run
1110111
Job Status: job has successfully run
0010111
Job Status: job has successfully run
1000011
Job Status: job has successfully run
0100011
Job Status: job has successfully run
1111001
Job Status: job has successfully run
1000111
Job Status: job has successfully run
0111101
Job Status: job has successfully run
1010101
Job Status: job has successfully run
0100001
Job Status: job has successfully run
1111000
Job Status: job is queued (None)  

In [ ]:
FNR = sum([list(tmp[0])[0:s].count("0") for tmp in res7_3])/s/100
FNRsd = np.std([list(tmp[0])[0:s].count("0") for tmp in res7_3])/s
FPR = sum([list(tmp[0])[s:p].count("1") for tmp in res7_3])/(p-s)/100
FPRsd = np.std([list(tmp[0])[s:p].count("1") for tmp in res7_3])/(p-s)
print("FNR is %f"%FNR)
print("sd of FNR is %f"%FNRsd)
print("FPR is %f"%FPR)
print("sd of FPR is %f"%FPRsd)